In [6]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn import datasets
import dask

In [7]:
## Set default figure size to be larger 
## this may only work in matplotlib 2.0+!
matplotlib.rcParams['figure.figsize'] = [10.0,6.0]
## Enable multiple outputs from jupyter cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
## Get Version information
print(sys.version)
print("Pandas version: {0}".format(pd.__version__))
print("Matplotlib version: {0}".format(matplotlib.__version__))
print("Numpy version: {0}".format(np.__version__))
print("SciKitLearn version: {0}".format(sklearn.__version__))
print("Dask version: {0}".format(dask.__version__))

3.6.7 |Anaconda custom (64-bit)| (default, Oct 23 2018, 14:01:38) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas version: 0.23.4
Matplotlib version: 3.0.2
Numpy version: 1.15.4
SciKitLearn version: 0.20.1
Dask version: 1.0.0


In [11]:
import os
print("My working director:\n" + os.getcwd())
os.chdir(r"/Users/wenhuizeng/Library/Mobile Documents/com~apple~CloudDocs/high performance/week12")
print("My working director:\n" + os.getcwd())

My working director:
/Users/wenhuizeng/Library/Mobile Documents/com~apple~CloudDocs/high performance/week13
My working director:
/Users/wenhuizeng/Library/Mobile Documents/com~apple~CloudDocs/high performance/week12


In [16]:
## Set print limits
pd.options.display.max_rows = 10
## Import Data
df_patient = pd.read_csv('PatientAnalyticFile.csv')
df_patient.head()
df_patient.columns

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2013-04-27,2018-06-01,NaN
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2011-11-05,2015-11-13,NaN
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2006-09-22,2018-06-01,NaN


Index(['PatientID', 'DateOfBirth', 'Gender', 'Race', 'Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease', 'Stroke',
       'Dementia', 'Pulmonary', 'Rheumatic', 'Peptic_ulcer_disease',
       'LiverMild', 'Diabetes_without_complications',
       'Diabetes_with_complications', 'Paralysis', 'Renal', 'Cancer',
       'LiverSevere', 'Metastatic_solid_tumour', 'HIV', 'Obesity',
       'Depression', 'Hypertension', 'Drugs', 'Alcohol',
       'First_Appointment_Date', 'Last_Appointment_Date', 'DateOfDeath'],
      dtype='object')

In [19]:
df_patient['death']=np.where(df_patient['DateOfDeath'].isnull(),0,1)
df_patient['death'].describe()

count    20000.000000
mean         0.354700
std          0.478434
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: death, dtype: float64

In [20]:
df_patient.describe()

,PatientID,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,Rheumatic,Peptic_ulcer_disease,LiverMild,...,Cancer,LiverSevere,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,death
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,0.045600,0.043450,0.023950,0.028650,0.031400,0.072650,0.012300,0.009650,0.009250,...,0.050450,0.051450,0.033150,0.006450,0.163450,0.106300,0.302900,0.040050,0.079750,0.354700
std,5773.647028,0.208621,0.203873,0.152897,0.166825,0.174401,0.259568,0.110224,0.097762,0.095733,...,0.218877,0.220919,0.179033,0.080054,0.369785,0.308229,0.459524,0.196081,0.270913,0.478434
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10000.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15000.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,20000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
df_patient.dtypes

PatientID                  int64
DateOfBirth               object
Gender                    object
Race                      object
Myocardial_infarction      int64
                           ...  
Alcohol                    int64
First_Appointment_Date    object
Last_Appointment_Date     object
DateOfDeath               object
death                      int64
Length: 30, dtype: object

In [32]:
df_patient['DateOfBirth']=pd.to_datetime(df_patient['DateOfBirth'])
df_patient['Age'] = ((pd.to_datetime('2019-04-21')-df_patient['DateOfBirth']).dt.days)/365.25
df_patient['Age'].describe()

count    20000.000000
mean        51.548638
std         18.145086
min         20.054757
25%         36.034908
50%         51.400411
75%         67.225188
max         83.044490
Name: Age, dtype: float64

In [38]:
vars_remove = ['PatientID','First_Appointment_Date','DateOfBirth',
               'Last_Appointment_Date','DateOfDeath','mortality']
vars_left = set(df_patient.columns)-set(vars_remove)
formula = 'death ~ ' + '+'.join(vars_left)
formula

'death ~ Dementia+Stroke+Obesity+Congestive_heart_failure+Diabetes_with_complications+death+Age+Race+LiverSevere+LiverMild+Hypertension+Peptic_ulcer_disease+Alcohol+Depression+Gender+Pulmonary+Myocardial_infarction+Cancer+Peripheral_vascular_disease+Drugs+Renal+Metastatic_solid_tumour+Rheumatic+Paralysis+HIV+Diabetes_without_complications'

In [42]:
df_patient_sub = df_patient.sample(frac=0.1,random_state=42)
Y,X =dmatrices(formula,df_patient_sub)

In [47]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,np.ravel(Y),test_size=0.25,random_state=42)

In [49]:
x_train.shape

(1500, 29)

In [50]:
x_test.shape

(500, 29)

In [51]:
y_train.shape

(1500,)

In [52]:
y_test.shape

(500,)

# Null Model

In [55]:
1-y_train.mean()

array(0.62933333)

In [57]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='most_frequent',random_state=0)
clf.fit(x_train,y_train)
clf.score(x_train,y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

array(0.62933333)

In [62]:
result_scores = {}
def get_results(x1):
    print("\n{0:20}   {1:4}    {2:4}".format('Model','Train','Test'))
    print('-------------------------------------------')
    for i in x1.keys():
        print("{0:20}   {1:<6.4}   {2:<6.4}".format(i,x1[i][0],x1[i][1]))

In [70]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Null                   0.6293   0.7   


In [73]:
result_scores['Null'] = (sklearn.metrics.accuracy_score(y_train,clf.predict(x_train)),
                        sklearn.metrics.accuracy_score(y_test,clf.predict(x_test)))

## Logistic Regression with the SAG Solver

* We will start with a logistic regression model, using the `solver=sag` option. 
* SAG handles an L2 penalty (not L1)
* For any SAG based approach, the model optimization is sensitive to the scale of the inputs. So we must scale our parameters on the way in! 
* We can use the pipeline approach to preprocess the data.

In [90]:
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

scaler = preprocessing.StandardScaler()

clf = linear_model.LogisticRegressionCV(solver='sag',fit_intercept=False,
                                       Cs=20,cv=5,penalty='l2',
                                       max_iter = 500,random_state=10)
pipe1 = Pipeline([('scaler',scaler),
                 ('logit',clf)])
pipe1.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logit', LogisticRegressionCV(Cs=20, class_weight=None, cv=5, dual=False,
           fit_intercept=False, intercept_scaling=1.0, max_iter=500,
           multi_class='warn', n_jobs=None, penalty='l2', random_state=10,
           refit=True, scoring=None, solver='sag', tol=0.0001, verbose=0))])

In [91]:
result_scores['logit_sag'] = (sklearn.metrics.accuracy_score(y_train,pipe1.predict(x_train)),
                        sklearn.metrics.accuracy_score(y_test,pipe1.predict(x_test)))

## Logistic Regression with the SAGA Solver

* We will start with a logistic regression model, using the `solver=saga` option. 
* SAGA handles an L1 penalty (not L2)
* For any SAG based approach, the model optimization is sensitive to the scale of the inputs. So we must scale our parameters on the way in! 
* We can use the pipeline approach to preprocess the data. 

In [93]:
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.pipeline import Pipeline

scaler = preprocessing.StandardScaler()

clf = linear_model.LogisticRegressionCV(solver='saga',penalty='l1',
                                       Cs=20,cv=5,fit_intercept=False,max_iter=500,random_state=10)

pipe2 = Pipeline([('scaler',scaler),('logit_saga',clf)])

pipe2.fit(x_train,y_train)


Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logit_saga', LogisticRegressionCV(Cs=20, class_weight=None, cv=5, dual=False,
           fit_intercept=False, intercept_scaling=1.0, max_iter=500,
           multi_class='warn', n_jobs=None, penalty='l1', random_state=10,
           refit=True, scoring=None, solver='saga', tol=0.0001, verbose=0))])

In [94]:
result_scores['logit_saga'] = (sklearn.metrics.accuracy_score(y_train,pipe2.predict(x_train)),
                        sklearn.metrics.accuracy_score(y_test,pipe2.predict(x_test)))

## Stochastic Gradient Descent in Scikit-learn

In [98]:
from sklearn.linear_model import SGDClassifier

scaler = preprocessing.StandardScaler()

clf =  linear_model.SGDClassifier(fit_intercept=False,
                                 loss='log',
                                 shuffle=True,
                                 verbose=1,
                                 random_state=12,
                                 max_iter=1000,
                                 tol=1e-6)
pipe3 = Pipeline([('scale',scaler),
                 ('sgd',clf)])

pipe3.fit(x_train,y_train)

-- Epoch 1
Norm: 197.27, NNZs: 28, Bias: 0.000000, T: 1500, Avg. loss: 1.887628
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 139.25, NNZs: 28, Bias: 0.000000, T: 3000, Avg. loss: 0.133229
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 105.18, NNZs: 28, Bias: 0.000000, T: 4500, Avg. loss: 0.050321
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 85.42, NNZs: 28, Bias: 0.000000, T: 6000, Avg. loss: 0.006401
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 70.35, NNZs: 28, Bias: 0.000000, T: 7500, Avg. loss: 0.000007
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 59.80, NNZs: 28, Bias: 0.000000, T: 9000, Avg. loss: 0.000002
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 52.00, NNZs: 28, Bias: 0.000000, T: 10500, Avg. loss: 0.000003
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 46.00, NNZs: 28, Bias: 0.000000, T: 12000, Avg. loss: 0.000002
Total training time: 0.00 seconds.
-- Epoch 9
Norm: 41.24, NNZs: 28, Bias: 0.000000, T: 13500, Avg. loss: 0.000003
Tot

Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sgd', SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=False,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=12, shuffle=True, tol=1e-06,
       validation_fraction=0.1, verbose=1, warm_start=False))])

In [102]:
## Score the Model on Training and Testing Set
result_scores['SAG'] = \
            (sklearn.metrics.accuracy_score(y_train,pipe3.predict(x_train)),
             sklearn.metrics.accuracy_score(y_test,pipe3.predict(x_test)))

In [104]:
from sklearn.linear_model import SGDClassifier

scaler = preprocessing.StandardScaler()

clf = linear_model.SGDClassifier(loss='log',fit_intercept=False,
                                penalty='elasticnet',
                                alpha=1,
                                shuffle=True,
                                verbose=1,
                                random_state=12,
                                max_iter=1000,
                                tol=1e-6)

pipe3 = Pipeline([('scaler',scaler),('sdg',clf)])

pipe3.fit(x_train,y_train)

-- Epoch 1
Norm: 0.39, NNZs: 2, Bias: 0.000000, T: 1500, Avg. loss: 0.552883
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 0.39, NNZs: 2, Bias: 0.000000, T: 3000, Avg. loss: 0.555532
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 0.39, NNZs: 2, Bias: 0.000000, T: 4500, Avg. loss: 0.554717
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 0.39, NNZs: 2, Bias: 0.000000, T: 6000, Avg. loss: 0.554792
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 0.39, NNZs: 2, Bias: 0.000000, T: 7500, Avg. loss: 0.554900
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 0.39, NNZs: 2, Bias: 0.000000, T: 9000, Avg. loss: 0.554792
Total training time: 0.01 seconds.
Convergence after 6 epochs took 0.01 seconds


Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sdg', SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=False, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=1000, n_iter=None,
       n_iter_no_change=5, n_jobs=None, penalty='elasticnet', power_t=0.5,
       random_state=12, shuffle=True, tol=1e-06, validation_fraction=0.1,
       verbose=1, warm_start=False))])

In [105]:
## Score the Model on Training and Testing Set
result_scores['SAG_regularize'] = \
            (sklearn.metrics.accuracy_score(y_train,pipe3.predict(x_train)),
             sklearn.metrics.accuracy_score(y_test,pipe3.predict(x_test)))

In [112]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

parameters = {'alpha':(0.001,0.01,0.5,1,2,5,10),
              'l1_ratio':(0,0.1,0.25,0.5,0.75,0.9,1)}

clf = linear_model.SGDClassifier(fit_intercept=False,
                                loss='log',
                                penalty='elasticnet',
                                shuffle=True,
                                random_state=12,
                                max_iter=1000,
                                tol=1e-6)

scaler = preprocessing.StandardScaler()

clf2 = GridSearchCV(clf,parameters,cv=5,return_train_score=True)

pipe4 = Pipeline([('scale',scaler),
                 ('sdg',clf2)])

pipe4.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sdg', GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=False,
       l1_ra...     pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0))])

In [117]:
## check best model params
pipe4.named_steps['sdg'].cv_results_

{'mean_fit_time': array([0.00513082, 0.00424204, 0.00477939, 0.00291638, 0.00259571,
        0.0026855 , 0.00259175, 0.00255618, 0.00280328, 0.00474625,
        0.00316377, 0.00419192, 0.00337014, 0.00315814, 0.00375938,
        0.0034636 , 0.00371437, 0.00326142, 0.00410542, 0.00394044,
        0.00235672, 0.0030848 , 0.00308018, 0.00304604, 0.00234785,
        0.00253463, 0.0025125 , 0.00249758, 0.00346508, 0.00277653,
        0.00232391, 0.0023458 , 0.00224085, 0.00230193, 0.00226235,
        0.00327501, 0.00246577, 0.00350637, 0.00356431, 0.00357528,
        0.0035048 , 0.00350819, 0.00469928, 0.0035708 , 0.00344481,
        0.00349135, 0.00346861, 0.00335822, 0.01938376]),
 'std_fit_time': array([3.60272844e-04, 6.75854001e-04, 1.19791106e-03, 2.60699150e-04,
        1.42087664e-04, 1.46023966e-04, 2.27754798e-04, 8.84453366e-05,
        1.14515936e-04, 2.44179762e-03, 3.32300607e-04, 5.46063754e-04,
        6.49094178e-04, 5.81705557e-04, 3.13069650e-04, 5.95686930e-04,
        6

In [119]:
pipe4.named_steps['sdg'].best_params_

{'alpha': 0.001, 'l1_ratio': 0}

In [120]:
## Score the Model on Training and Testing Set
result_scores['SAG_en_cv'] = \
            (sklearn.metrics.accuracy_score(y_train,pipe4.predict(x_train)),
             sklearn.metrics.accuracy_score(y_test,pipe4.predict(x_test)))

In [121]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Null                   0.6293   0.7   
logit_sag              1.0      0.998 
logit_saga             1.0      1.0   
SAG                    1.0      1.0   
SAG_regularize         1.0      1.0   
SAG_en_cv              1.0      1.0   


# Streaming the data

In [122]:
df_patient_gen = pd.read_csv(r'PatientAnalyticFile.csv',chunksize=1000)
for n,chunk in enumerate(df_patient_gen):
    print(n)
    print(chunk)
    if n>1:
        break

0
     PatientID DateOfBirth  Gender      Race  Myocardial_infarction  \
0            1  1962-02-27  female  hispanic                      0   
1            2  1959-08-18    male     white                      0   
2            3  1946-02-15  female     white                      0   
3            4  1979-07-27  female     white                      0   
4            5  1983-02-19  female  hispanic                      0   
..         ...         ...     ...       ...                    ...   
995        996  1964-07-28  female     black                      0   
996        997  1980-10-07    male     white                      0   
997        998  1949-09-06    male     black                      0   
998        999  1945-07-20    male     white                      0   
999       1000  1987-04-14    male     white                      0   

     Congestive_heart_failure  Peripheral_vascular_disease  Stroke  Dementia  \
0                           0                            0       

In [155]:
df1 = pd.read_csv(r'PatientAnalyticFile.csv',nrows=10)
df1['mortality'] =  np.where(df1['DateOfDeath'].isnull(),0,1)
df1['DateOfBirth'] = pd.to_datetime(df1['DateOfBirth'])
df1['Age_years'] = ((pd.to_datetime('2019-04-21')-df1['DateOfBirth']).dt.days/365.25)
df1

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath,mortality,Age_years
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,2013-04-27,2018-06-01,NaN,0,57.144422
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02,1,59.674196
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,0,1,0,0,2011-11-05,2015-11-13,NaN,0,73.177276
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17,1,39.734428
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,1,0,0,2006-09-22,2018-06-01,NaN,0,36.167009
5,6,1987-11-16,male,black,0,0,0,0,0,0,...,0,0,0,0,0,2006-10-22,2011-01-13,NaN,0,31.427789
6,7,1958-01-11,male,white,0,0,0,0,0,0,...,0,0,0,0,0,2015-01-20,2018-06-01,NaN,0,61.273101
7,8,1952-10-31,female,black,0,0,0,0,0,0,...,0,0,1,0,0,2013-03-25,2018-06-01,NaN,0,66.469541
8,9,1951-10-06,female,white,0,0,0,0,0,0,...,0,1,0,0,0,2008-08-04,2010-05-23,NaN,0,67.540041
9,10,1954-10-16,male,white,0,0,0,0,0,0,...,0,0,0,0,0,2014-07-01,2015-10-19,NaN,0,64.511978


In [157]:
## standard numeric columns

num_cols = ['Age_years']

## categorical columns

cat_cols = ['Gender', 'Race', 'Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease', 'Stroke',
       'Dementia', 'Pulmonary', 'Rheumatic', 'Peptic_ulcer_disease',
       'LiverMild', 'Diabetes_without_complications',
       'Diabetes_with_complications', 'Paralysis', 'Renal', 'Cancer',
       'LiverSevere', 'Metastatic_solid_tumour', 'HIV', 'Obesity',
       'Depression', 'Hypertension', 'Drugs', 'Alcohol','mortality']

def sum2(x):
    return(np.sum(x**2))
df1.loc[:,num_cols].agg(sum2,axis=0)
df1.loc[:,num_cols].agg(np.sum,axis=0)

Age_years    32952.036793
dtype: float64

Age_years    557.119781
dtype: float64

In [158]:
dict_unique = {}
for col in cat_cols:
    dict_unique[col] = set(df1.loc[:,col])
dict_unique
dict_unique['Gender']

{'Gender': {'female', 'male'},
 'Race': {'black', 'hispanic', 'white'},
 'Myocardial_infarction': {0},
 'Congestive_heart_failure': {0},
 'Peripheral_vascular_disease': {0},
 'Stroke': {0},
 'Dementia': {0},
 'Pulmonary': {0, 1},
 'Rheumatic': {0},
 'Peptic_ulcer_disease': {0, 1},
 'LiverMild': {0},
 'Diabetes_without_complications': {0, 1},
 'Diabetes_with_complications': {0},
 'Paralysis': {0},
 'Renal': {0},
 'Cancer': {0, 1},
 'LiverSevere': {0, 1},
 'Metastatic_solid_tumour': {0},
 'HIV': {0, 1},
 'Obesity': {0},
 'Depression': {0, 1},
 'Hypertension': {0, 1},
 'Drugs': {0},
 'Alcohol': {0},
 'mortality': {0, 1}}

{'female', 'male'}

In [161]:
df_patient_gen2 = pd.read_csv(r'PatientAnalyticFile.csv',chunksize=100)
for n, chunk in enumerate(df_patient_gen2):
    chunk['mortality']=np.where(chunk['DateOfDeath'].isnull(),0,1)
    chunk['DateOfBirth'] = pd.to_datetime(chunk['DateOfBirth'])
    chunk['Age_years'] =  ((pd.to_datetime('2019-04-22')-chunk['DateOfBirth']).dt.days/365.25)
    if n==0:
        n_rows=chunk.shape[0]
        running_sum=chunk.loc[:,num_cols].agg(np.sum,axis=0)
        running_sum_2 =  chunk.loc[:,num_cols].agg(sum2,axis=0)
        dict_unique={}
        for col in cat_cols:
            dict_unique[col] = set(chunk.loc[:,col])
    if n>0:
        n_rows+=chunk.shape[0]
        running_sum =  running_sum+chunk.loc[:,num_cols].agg(np.sum,axis=0)
        running_sum_2 +=chunk.loc[:,num_cols].agg(sum2,axis=0)
        for col in cat_cols:
            dict_unique[col]=set(chunk.loc[:,col]).union(set(dict_unique[col]))

In [164]:
num_cols
n_rows
running_sum

['Age_years']

20000

Age_years    1.031028e+06
dtype: float64

In [165]:
mean_all = running_sum/n_rows
stdev_all = np.sqrt((running_sum_2/n_rows)-mean_all*mean_all)
mean_all
stdev_all

Age_years    51.551375
dtype: float64

Age_years    18.144632
dtype: float64

In [131]:
## calculate mean
mean_all = running_sum / n_rows
## calculate stdev
stdev_all = np.sqrt((running_sum_2 / n_rows) - (mean_all * mean_all)) 
## final nums to use for standardization:
mean_all
stdev_all

Age_years    47.247474
dtype: float64

Age_years    18.144632
dtype: float64

In [133]:
df2 = pd.read_csv(r'PatientAnalyticFile.csv', nrows=10)
# Create mortality variable
df2['mortality'] = \
    np.where(df2['DateOfDeath'].isnull(), 0,1)

# Convert dateofBirth to date
df2['DateOfBirth'] = \
    pd.to_datetime(df2['DateOfBirth'])
# Calculate age in years as of 2015-01-01
df2['Age_years'] = \
    ((pd.to_datetime('2015-01-01') - df2['DateOfBirth']).dt.days/365.25)
df2

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath,mortality,Age_years
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,2013-04-27,2018-06-01,NaN,0,52.843258
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02,1,55.373032
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,0,1,0,0,2011-11-05,2015-11-13,NaN,0,68.876112
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17,1,35.433265
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,1,0,0,2006-09-22,2018-06-01,NaN,0,31.865845
5,6,1987-11-16,male,black,0,0,0,0,0,0,...,0,0,0,0,0,2006-10-22,2011-01-13,NaN,0,27.126626
6,7,1958-01-11,male,white,0,0,0,0,0,0,...,0,0,0,0,0,2015-01-20,2018-06-01,NaN,0,56.971937
7,8,1952-10-31,female,black,0,0,0,0,0,0,...,0,0,1,0,0,2013-03-25,2018-06-01,NaN,0,62.168378
8,9,1951-10-06,female,white,0,0,0,0,0,0,...,0,1,0,0,0,2008-08-04,2010-05-23,NaN,0,63.238877
9,10,1954-10-16,male,white,0,0,0,0,0,0,...,0,0,0,0,0,2014-07-01,2015-10-19,NaN,0,60.210815


In [166]:
## standardized numeric cols
X_num=(df2.loc[:,num_cols]-mean_all)/stdev_all
X_num

,Age_years
0,0.071199
1,0.210622
2,0.954813
3,-0.888313
4,-1.084923
5,-1.346114
6,0.298742
7,0.585132
8,0.644130
9,0.477245


In [167]:
for col in cat_cols:
    df2[col]=pd.Categorical(df2[col],categories=dict_unique[col])
    
formula = 'mortality ~ ' + '+'.join(cat_cols)
formula

'mortality ~ Gender+Race+Myocardial_infarction+Congestive_heart_failure+Peripheral_vascular_disease+Stroke+Dementia+Pulmonary+Rheumatic+Peptic_ulcer_disease+LiverMild+Diabetes_without_complications+Diabetes_with_complications+Paralysis+Renal+Cancer+LiverSevere+Metastatic_solid_tumour+HIV+Obesity+Depression+Hypertension+Drugs+Alcohol+mortality'

In [171]:
Y,X_cat =  dmatrices(formula,df2)
X_all = np.concatenate([X_cat,X_num],axis=1)


DesignMatrix with shape (10, 2)
  mortality[0]  mortality[1]
             1             0
             0             1
             1             0
             0             1
             1             0
             1             0
             1             0
             1             0
             1             0
             1             0
  Terms:
    'mortality' (columns 0:2)

In [173]:
X_all=np.concatenate([X_cat,X_num],axis=1)

In [174]:
Y[:,1]
classes_potential =  np.array([0,1])
classes_potential

array([0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])

array([0, 1])

In [180]:
classes_potential =  np.array([0,1])
clf =  linear_model.SGDClassifier(fit_intercept=False,
                                 loss='log',
                                 penalty='elasticnet',
                                 warm_start=True,
                                 alpha=1,
                                 verbose=1,
                                 tol=1e-6)
## establish generator to yield data
df_patient_gen3 = \
 pd.read_csv(r'PatientAnalyticFile.csv',
             chunksize=500)

for n, chunk in enumerate(df_patient_gen3):
    chunk['mortality'] = np.where(chunk['DateOfDeath'].isnull(),0,1)
    chunk['DateOfBirth'] = pd.to_datetime(chunk['DateOfBirth'])
    chunk['Age_years'] = (pd.to_datetime('2019-04-22')-chunk['DateOfBirth']).dt.days/365.25
    X_num=(chunk.loc[:,num_cols]-mean_all)/stdev_all
    for col in cat_cols:
        chunk[col] = pd.Categorical(chunk[col],categories=dict_unique[col])
    Y,X_cat = dmatrices(formula,chunk)
    X_all = np.concatenate([X_cat,X_num],axis=1)
    clf.partial_fit(X_all,Y[:,1],classes=classes_potential)
    

-- Epoch 1
Norm: 0.13, NNZs: 2, Bias: 0.000000, T: 500, Avg. loss: 0.680285
Total training time: 0.00 seconds.


SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=False, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=None, penalty='elasticnet', power_t=0.5,
       random_state=None, shuffle=True, tol=1e-06, validation_fraction=0.1,
       verbose=1, warm_start=True)

-- Epoch 1
Norm: 0.11, NNZs: 3, Bias: 0.000000, T: 500, Avg. loss: 0.678611
Total training time: 0.00 seconds.


SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=False, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=None, penalty='elasticnet', power_t=0.5,
       random_state=None, shuffle=True, tol=1e-06, validation_fraction=0.1,
       verbose=1, warm_start=True)

-- Epoch 1
Norm: 0.10, NNZs: 3, Bias: 0.000000, T: 500, Avg. loss: 0.677224
Total training time: 0.00 seconds.


...

SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=False, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=None, penalty='elasticnet', power_t=0.5,
       random_state=None, shuffle=True, tol=1e-06, validation_fraction=0.1,
       verbose=1, warm_start=True)

-- Epoch 1
Norm: 0.07, NNZs: 3, Bias: 0.000000, T: 500, Avg. loss: 0.678963
Total training time: 0.00 seconds.


SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=False, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=None, penalty='elasticnet', power_t=0.5,
       random_state=None, shuffle=True, tol=1e-06, validation_fraction=0.1,
       verbose=1, warm_start=True)

-- Epoch 1
Norm: 0.07, NNZs: 3, Bias: 0.000000, T: 500, Avg. loss: 0.674716
Total training time: 0.00 seconds.


SGDClassifier(alpha=1, average=False, class_weight=None, early_stopping=False,
       epsilon=0.1, eta0=0.0, fit_intercept=False, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_iter_no_change=5, n_jobs=None, penalty='elasticnet', power_t=0.5,
       random_state=None, shuffle=True, tol=1e-06, validation_fraction=0.1,
       verbose=1, warm_start=True)

In [ ]:
## test the loop!
for n,chunk in enumerate(df_patient_gen3):
    ## process data first
    # Create mortality variable
    chunk['mortality'] = \
        np.where(chunk['DateOfDeath'].isnull(),
                 0,1)
    # Convert dateofBirth to date
    chunk['DateOfBirth'] = \
        pd.to_datetime(chunk['DateOfBirth'])
    # Calculate age in years as of 2015-01-01
    chunk['Age_years'] = \
        ((pd.to_datetime('2019-04-22') - chunk['DateOfBirth']).dt.days/365.25)
    # standardized numeric cols
    X_num = (chunk.loc[:,num_cols] - mean_all)/stdev_all
    ## get dummy coded categoricals
    for col in cat_cols:
        chunk[col] = pd.Categorical(chunk[col],
                                    categories=dict_unique[col])
    ## use Patsy to create model matrices
    Y,X_cat = dmatrices(formula,
                        chunk)
    ## join in the continuous X
    X_all = np.concatenate([X_cat,X_num],axis=1)
    
    ## update the fit, one full EPOCH?
    clf.partial_fit(X_all,Y[:,1],classes=classes_potential)

